In [8]:
import gradio as gr
from gradio import components
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

logo_html = """
<div style="position: absolute; top: 10px; right: 10px;">
    <img src="templates/img/TUM_Logo_extern_mt_DE_RGB_j.jpg" width="100" height="100">
</div>
"""

def submit_form(hypothesis, material, implementation, observation, results, agreement):

    hypothesis_custom = f"The hypothesis of my experiment is: {hypothesis}, please check and evaluate it."
    material_custom = f"The material of my experiment is: {material}, please check and evaluate it."
    implementation_custom = f"The implementation of my experiment is: {implementation}, please check and evaluate it."
    observation_custom = f"The observation of my experiment is: {observation}, please check and evaluate it."
    results_custom = f"The results of my experiment is: {results}, please check and evaluate it."

    hypothesis_token = tokenizer.encode(hypothesis_custom, return_tensors="pt")
    material_token = tokenizer.encode(material_custom, return_tensors="pt")
    implementation_token = tokenizer.encode(implementation_custom, return_tensors="pt")
    observation_token = tokenizer.encode(observation_custom, return_tensors="pt")
    results_token = tokenizer.encode(results_custom, return_tensors="pt")
    
    hypothesis_output = model.generate(hypothesis_token, max_length=100, num_return_sequences=1, repetition_penalty=1.5)
    material_output = model.generate(material_token, max_length=100, num_return_sequences=1, repetition_penalty=1.5)
    implementation_output = model.generate(implementation_token, max_length=100, num_return_sequences=1, repetition_penalty=1.5)
    observation_output = model.generate(observation_token, max_length=100, num_return_sequences=1, repetition_penalty=1.5)
    results_output = model.generate(results_token, max_length=100, num_return_sequences=1, repetition_penalty=1.5)

    hypothesis_eval = tokenizer.decode(hypothesis_output[0], skip_special_tokens=True)
    material_eval = tokenizer.decode(material_output[0], skip_special_tokens=True)  
    implementation_eval = tokenizer.decode(implementation_output[0], skip_special_tokens=True)
    observation_eval = tokenizer.decode(observation_output[0], skip_special_tokens=True)
    results_eval = tokenizer.decode(results_output[0], skip_special_tokens=True)

    if agreement:
        feedback = f"Form submitted successfully!\n\n" \
                   f"Hypothesis: {hypothesis_eval}\n" \
                   f"Material: {material_eval}\n" \
                   f"Implementation: {implementation_eval}\n" \
                   f"Observation: {observation_eval}\n" \
                   f"Results: {results_eval}"
    else:
        feedback = "Please agree to the data protection agreement before submitting the form."

    return feedback

form_inputs = [
    components.Textbox(label="Hypothesis"),
    components.Textbox(label="Material"),
    components.Textbox(label="Implementation"),
    components.Textbox(label="Observation"),
    components.Textbox(label="Results"),
]

form_output = components.Textbox(label="Feedback")

agreement_checkbox = components.Checkbox(label="I agree to the data protection agreement.")

description_html = "<div style='text-align: center;'>Empower Your Experiments, Enhance Your Learning</div>"

iface = gr.Interface(fn=submit_form, inputs=form_inputs + [agreement_checkbox], outputs=form_output,
                     title="ExperimentEvaluator", description=f"{logo_html}\n{description_html}")

iface.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
